#Archivo para limpieza de los datos generados por el SAMTM

El esquema de la base de datos necesario es el siguiente:
  - id_evento
  - operador [sin desarrollar]
  - turno [sin desarrollar]
  - maquina
  - causa
  - inicio
  - fin
  - duración [seg]

## Librerías

In [235]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Cargar datos
Para poder cargar los datos deben de estar en el mismo entorno de ejecución

El esquema de los datos se presenta de la siguiente forma:

  P2:

    - fecha, Formato DD/MM/AAAA
    - hora, Formato HH:MM:SS
    - Maquina 1,
    - Maquina 2, estados de las máquina [1,2,3]
    - Maquina 3,
    - Maquina 4:10, no utilizados

  Los estados de las máquinas corresponden:

    1- Producción
    2- Paro
    3- Preparación

  P3:

    - fecha, Formato DD/MM/AAAA
    - hora, Formato HH:MM:SS
    - Maquina 1,
    - Maquina 2, en este campo se captura el id de causa de paro
    - Maquina 3,
    - Maquina 4:10, no utilizados


El archivo "catalogoCausas.csv" muestra la base de causas con su id correspondiente.

In [236]:
# Cargar datos, se creará un df por cada .csv importado -----

dataP2 = pd.read_csv("P2.csv")
dataP3 = pd.read_csv("P3.csv")
causas = pd.read_csv("catalogoCausas.csv")


## 2. Limpiar datos
Consta de tres etapas:
  1. Eliminar columnas innecesarias.
  2. Detectar cambios de estado y eliminar eventos sin cambios.
  3. Calcular duración del evento.

In [237]:
# Crear df de datos limpios con su estructura según el esquema [turno y operador aún no se agregan]

dfClean = {"maquina":[],
           "causa":[],
           "inicio":[],
           "fin":[],
           "duracion":[]}
dfClean = pd.DataFrame(dfClean)

In [238]:
# Unificar columnas de fecha y hora, formatear a datetime -----
# Tabla P2

dataP2['marcaTemporal'] = pd.to_datetime(dataP2['fecha'] + " " + dataP2['hora'])
dataP2 = dataP2[['marcaTemporal', 'Maquina 1', 'Maquina 2', 'Maquina 3']]

In [239]:
# Copiar las causas de paro para mantener un solo df

cols = ["Maquina 1", "Maquina 2", "Maquina 3"]

dataP2[["causaMaquina_1", "causaMaquina_2", "causaMaquina_3"]] = dataP3[cols].rename(
    columns={
       "Maquina 1" : "causaMaquina_1",
       "Maquina 2" : "causaMaquina_2",
       "Maquina 3" : "causaMaquina_3"
    }
)

# Renombrar las columnas para mantener el formato y limpieza

dataP2 = dataP2.rename(
    columns={
       "Maquina 1" : "maquina_1",
       "Maquina 2" : "maquina_2",
       "Maquina 3" : "maquina_3"
    }
)

In [240]:
# Detectar cambios de estado -----

for i in ["maquina_1", "maquina_2", "maquina_3"]:
  dataP2[f"cambio_{i[-1]}"] = dataP2[i].ne(dataP2[i].shift())

# Mantener solo los eventos donde hubo al menos un cambio -----

dataP2 = dataP2[dataP2[['cambio_1','cambio_2','cambio_3']].any(axis=1)].copy()

# Calcular duración hacia el siguiente evento -----

dataP2["siguienteMarca"] = dataP2["marcaTemporal"].shift(-1)
dataP2["duracion"] = (dataP2["siguienteMarca"] - dataP2["marcaTemporal"]) #/pd.Timedelta(seconds=1)

dataP2.reset_index(drop=True, inplace=True)


In [241]:
dataP2

,marcaTemporal,maquina_1,maquina_2,maquina_3,causaMaquina_1,causaMaquina_2,causaMaquina_3,cambio_1,cambio_2,cambio_3,siguienteMarca,duracion
0,2025-12-12 07:23:10,2,2,2,0,0,0,True,True,True,2025-12-12 07:24:11,0 days 00:01:01
1,2025-12-12 07:24:11,2,3,2,0,26,0,False,True,False,2025-12-12 07:27:24,0 days 00:03:13
2,2025-12-12 07:27:24,2,3,3,0,26,57,False,False,True,2025-12-12 07:36:08,0 days 00:08:44
3,2025-12-12 07:36:08,3,3,3,32,26,57,True,False,False,2025-12-12 07:39:57,0 days 00:03:49
4,2025-12-12 07:39:57,2,1,1,0,0,0,True,True,True,2025-12-12 07:43:05,0 days 00:03:08
5,2025-12-12 07:43:05,3,1,1,32,0,0,True,False,False,2025-12-12 07:44:09,0 days 00:01:04
6,2025-12-12 07:44:09,1,1,1,0,0,0,True,False,False,2025-12-12 07:44:39,0 days 00:00:30
7,2025-12-12 07:44:39,2,1,1,0,0,0,True,False,False,2025-12-12 07:47:53,0 days 00:03:14
8,2025-12-12 07:47:53,3,1,1,32,0,0,True,False,False,2025-12-12 07:49:25,0 days 00:01:32
9,2025-12-12 07:49:25,1,1,1,0,0,0,True,False,False,2025-12-12 07:52:37,0 days 00:03:12


In [250]:
estados = (
    dataP2.melt(
        id_vars=['marcaTemporal', 'duracion'],
        value_vars=['maquina_1', 'maquina_2', 'maquina_3'],
        var_name='maquina',
        value_name='estado'
    )
)

causas = (
    dataP2.melt(
        id_vars=['marcaTemporal', 'duracion'],
        value_vars=['causaMaquina_1', 'causaMaquina_2', "causaMaquina_3"],
        var_name='maquina',
        value_name='causa'
    )
)

causas = causas["causa"]

estados.join(causas)

,marcaTemporal,duracion,maquina,estado,causa
0,2025-12-12 07:23:10,0 days 00:01:01,maquina_1,2,0
1,2025-12-12 07:24:11,0 days 00:03:13,maquina_1,2,0
2,2025-12-12 07:27:24,0 days 00:08:44,maquina_1,2,0
3,2025-12-12 07:36:08,0 days 00:03:49,maquina_1,3,32
4,2025-12-12 07:39:57,0 days 00:03:08,maquina_1,2,0
...,...,...,...,...,...
76,2025-12-12 08:42:03,0 days 00:04:13,maquina_3,1,0
77,2025-12-12 08:46:16,0 days 00:04:43,maquina_3,1,0
78,2025-12-12 08:50:59,0 days 00:05:52,maquina_3,1,0
79,2025-12-12 08:56:51,0 days 00:01:33,maquina_3,1,0


In [249]:
causas

,causa
0,0
1,0
2,0
3,32
4,0
...,...
76,0
77,0
78,0
79,0


In [ ]:
estados

In [ ]:
dataP2_long = (
    dataP2_long
    .sort_values(['maquina', 'marcaTemporal'])
    .loc[lambda x: x['estado'].ne(x.groupby('maquina')['estado'].shift())]
    .reset_index(drop=True)
)

In [ ]:
dataP2_long

In [ ]:
mapa_estados = {
    1: 'Producción',
    2: 'Paro',
    3: 'Preparación'
}

dataP2_long['estado'] = dataP2_long['estado'].map(mapa_estados)

In [ ]:
dataP2_long

In [243]:
dataP2_long['fin'] = dataP2_long['marcaTemporal'].shift(-1)

dataP2_long.dropna(inplace=True)

In [244]:
dataP2_long

,marcaTemporal,duracion,maquina,estado,fin
0,2025-12-12 07:23:10,0 days 00:01:01,maquina_1,Paro,2025-12-12 07:36:08
1,2025-12-12 07:36:08,0 days 00:03:49,maquina_1,Preparación,2025-12-12 07:39:57
2,2025-12-12 07:39:57,0 days 00:03:08,maquina_1,Paro,2025-12-12 07:43:05
3,2025-12-12 07:43:05,0 days 00:01:04,maquina_1,Preparación,2025-12-12 07:44:09
4,2025-12-12 07:44:09,0 days 00:00:30,maquina_1,Producción,2025-12-12 07:44:39
5,2025-12-12 07:44:39,0 days 00:03:14,maquina_1,Paro,2025-12-12 07:47:53
6,2025-12-12 07:47:53,0 days 00:01:32,maquina_1,Preparación,2025-12-12 07:49:25
7,2025-12-12 07:49:25,0 days 00:03:12,maquina_1,Producción,2025-12-12 08:05:09
8,2025-12-12 08:05:09,0 days 00:03:42,maquina_1,Paro,2025-12-12 08:42:03
9,2025-12-12 08:42:03,0 days 00:04:13,maquina_1,Preparación,2025-12-12 08:46:16


## 4. Convertir a .csv

In [245]:
#dfClean["marcaTemporal"] = dfClean["marcaTemporal"].dt.strftime("%Y%m%d%H%M%S")

#dfClean["siguienteMarca"] = dfClean["siguienteMarca"].dt.strftime("%Y%m%d%H%M%S")

#dfClean["duracion"] = dfClean["duracion"].dt.strftime("%Y%m%d%H%M%S")

In [246]:
#dfClean.to_csv("datos.csv", index=False)